In [1]:
%matplotlib inline
%reset
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.patches as patches
from matplotlib.ticker import FormatStrFormatter
from matplotlib import pyplot as plt, animation
from math import sqrt, pi, cos

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Files and paths

In [2]:
path = "D:\\OceanParcels"
final_path = path + "\\data\\final grids\\final_" # final grids
fig_path = path + "\\outputs\\figures_2\\"
coords = path + "\\data\\coordinates.nc"

scenario = np.array(["S0", "S1", "S2", "S3", "S4", "S5", "S6"])
mon_2 = np.array(["may", "jun", "jul", "aug", "sep", "oct"])

## Basic stuff

In [3]:
dsc = xr.open_dataset(coords, decode_times = False).load()
dsc.close()

botz = dsc.botz.values*-1
longitudes = dsc.lon_grid.values[0,:]
latitudes = dsc.lat_grid.values[:,0]

del dsc

## Hydro data

Loads up all the data previously generated and saved.

In [4]:
from matplotlib.ticker import FormatStrFormatter
fmt = lambda x, pos: '{:.2f}'.format(x, pos)

#longitudes = longitudes[:41]
#latitudes = latitudes[:40]

maxUVW = 0.25
minUVW = 0.0

# Some pretty plots

These are just to get a look at what the flow fields look like.

In [ ]:
###### plt.rcParams["figure.figsize"] = [7, 7]
plt.rcParams["figure.autolayout"] = True

runs = ['aug_S4', 'aug_S5', 'aug_S6', 'sep_S0', 'sep_S1', 'sep_S2', 'sep_S3', 'sep_S4', 'sep_S5', 'sep_S6', 
        'oct_S0', 'oct_S1', 'oct_S2', 'oct_S3', 'oct_S4', 'oct_S5', 'oct_S6']

for r in range(17):
    runname = runs[r] #mon_2[mo] + "_" + scenario[sc]
    ds1name = final_path + runname + "_1.nc"
    print("Starting ", ds1name)

    # ds1 contains the data
    ds1 = xr.open_dataset(ds1name, decode_times = False).load()
    ds1.close()

    tt = sum(np.shape(ds1.times_secs.values))

    U_mod = ds1.U_atten.values
    V_mod = ds1.V_atten.values
    W = ds1.W.values

    slice_U = np.nanmean(U_mod[:,4:10,0:41,0:42], axis = 1)
    slice_V = np.nanmean(V_mod[:,4:10,0:41,0:42], axis = 1)
    slice_W = np.nanmean(W[:,4:10,0:41,0:42], axis = 1)
    canopy_UVW = np.sqrt(np.power(slice_U,2) + np.power(slice_V,2) + np.power(slice_W,2))

    # Set up plotting
    fig = plt.figure()
    ax = plt.axes() 
    # Animation function
    def animate(i): 
        z = canopy_UVW[i,:,:]
        col = plt.pcolormesh(longitudes, latitudes, z, vmin = minUVW, vmax = maxUVW)
        return col

    anim = matplotlib.animation.FuncAnimation(fig, animate, interval=6, frames = int(round(tt/6)))
    # All the formatting
    yticks = ax.get_ylim()
    xticks = ax.get_xlim()
    ax.set_yticks((np.linspace(yticks[0]+0.002, yticks[1]-0.002, 5)))
    ax.set_xticks((np.linspace(xticks[0]+0.005, xticks[1]-0.005, 5)))
    ax.xaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(fmt))
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(fmt))
    ax.set_ylabel("Latitude")
    ax.set_xlabel("Longitude")
    #plt.colorbar(col, label = "Relative attenuation")

    anim.save(fig_path + "canopy_UVW_" + runname + ".gif")
    plt.close()